<a href="https://colab.research.google.com/github/YKG-123/Fantasy_football_ML_Random_forest_Regressor/blob/main/WR_Prev_Season_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
import requests
from bs4 import BeautifulSoup
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from scipy.stats import pearsonr, spearmanr
import kagglehub
from kagglehub import KaggleDatasetAdapter
import statsmodels.api as sm


In [5]:
file_path = "weekly_player_stats_offense.csv"
file_path2 = "yearly_player_stats_offense.csv"

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "philiphyde1/nfl-stats-1999-2022",
  file_path,
)
df2 = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "philiphyde1/nfl-stats-1999-2022",
  file_path2,
)
import requests

years = range(2016, 2024)
all_adp = []

for y in years:
    url = f"https://fantasyfootballcalculator.com/api/v1/adp/ppr?teams=12&year={y}&position=all"
    try:
        r = requests.get(url)
        r.raise_for_status()
        data = r.json()

        df_adp = pd.DataFrame(data["players"])
        df_adp["season"] = y
        all_adp.append(df_adp)
        print(f"{y} — {len(df_adp)} players loaded")
    except Exception as e:
        print(f"Error for {y}: {e}")

adp_all = pd.concat(all_adp, ignore_index=True)

adp_all["name"] = adp_all["name"].str.strip()

print(adp_all.head())


adp_keep = adp_all[["name", "position", "team", "adp", "season"]].copy()

adp_keep["player_name_merge"] = (
    adp_keep["name"]
    .str.strip()
    .str.replace(r"\s+Jr\.?$", "", regex=True)
    .str.replace(r"\s+Sr\.?$", "", regex=True)
)

adp_all = (
    adp_keep
    .groupby(["season", "player_name_merge"], as_index=False)
    .agg({
        "adp": "first",
        }))

df["player_name_merge"] = (
    df["player_name"]
    .str.strip()
    .str.replace(r"\s+Jr\.?$", "", regex=True)
    .str.replace(r"\s+Sr\.?$", "", regex=True)
)

df= df.merge(adp_all, on=["season", "player_name_merge"], how="left")

print(df[["player_name", "season", "adp", "position", "team"]].head())

/tmp/ipython-input-2986080412.py:4: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'nfl-stats-1999-2022' dataset.


/tmp/ipython-input-2986080412.py:9: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df2 = kagglehub.load_dataset(


Using Colab cache for faster access to the 'nfl-stats-1999-2022' dataset.
2016 — 188 players loaded
2017 — 184 players loaded
2018 — 197 players loaded
2019 — 200 players loaded
2020 — 203 players loaded
2021 — 211 players loaded
2022 — 157 players loaded
2023 — 202 players loaded
   player_id               name position team  adp adp_formatted  \
0       1886      Antonio Brown       WR  PIT  1.2          1.01   
1       1796        Julio Jones       WR  ATL  2.8          1.03   
2       2113  Odell Beckham Jr.       WR  NYG  3.4          1.03   
3       2297      David Johnson       RB  ARI  3.4          1.03   
4       2280        Todd Gurley       RB  LAR  6.3          1.06   

   times_drafted  high  low  stdev  bye  season  
0            454     1    4    0.5    5    2016  
1            348     1    6    0.9    5    2016  
2            512     1   10    1.2   14    2016  
3            146     1    7    1.5    8    2016  
4            470     1   12    1.8    8    2016  
     play

In [ ]:
df=df[df["season"]>=2016]
df=df[df["season"]<=2022]
print(df["adp"])
df2=df2[df2["position"]=="WR"]
df2=df2[df2["season"]>=2016]
df2=df2[df2["season"]<=2022]
df2=df2[df2["season_average_fantasy_points_ppr"]>5]
df2["season_average_fantasy_points_ppr"].describe()

In [ ]:


df = df[df['position'] == 'WR'].copy()

df['week'] = df['week'].astype(int)
df['season'] = df['season'].astype(int)
df["adp"]=df["adp"].replace(0,200)

if 'fantasy_points_ppr' in df.columns:
    df['fantasy_points_ppr'] = df['fantasy_points_ppr'].astype(float)
    fantasy_points_col = 'fantasy_points_ppr'
elif 'season_average_fantasy_points_ppr' in df.columns:
     df['season_average_fantasy_points_ppr'] = df['season_average_fantasy_points_ppr'].astype(float)
     fantasy_points_col = 'season_average_fantasy_points_ppr'
else:
    raise KeyError("No suitable fantasy points column found in the dataframe.")


numeric_cols = df.select_dtypes(include='number').columns.tolist()
group_cols = ['player_id', 'season']
agg_cols = {c: 'mean' for c in numeric_cols if c not in group_cols and c != "fantasy_points_ppr" and c != "draft_" and c != "age" and c != "adp"}
agg_cols['fantasy_points_ppr'] = 'mean'
agg_cols['draft_ovr'] = 'first'
agg_cols['age'] = 'first'
agg_cols['adp'] = 'first'


season_agg = df.groupby(group_cols).agg(
    agg_cols
).reset_index()
if 'player_name' in df.columns:
    names = df.groupby(group_cols)['player_name'].first().reset_index()
    season_agg = season_agg.merge(names[['player_id', 'season', 'player_name']], on=['player_id', 'season'], how='left')

prev = season_agg.copy()
prev['season'] = prev['season'] + 1
prev = prev.add_suffix('_prev')

curr = season_agg.copy()
merged_prev_next = curr.merge(
    prev,
    left_on=['player_id', 'season'],
    right_on=['player_id_prev', 'season_prev'],
    how='inner' )

features_real = [      'fantasy_points_ppr_prev', 'adp',
            "receiving_yards_prev","draft_ovr",
            "targets_prev","age",
           "first_down_pass_prev"

]

if 'fantasy_points_ppr' in merged_prev_next.columns:
    target = 'fantasy_points_ppr'
elif fantasy_points_col in merged_prev_next.columns:
    target = fantasy_points_col
else:
    raise KeyError("No suitable target column found in the merged dataframe.")


model_df = merged_prev_next.dropna(subset=features_real + [target]).copy()
X = model_df[features_real]
X["adp"] = X["adp"].replace(0, 200)

y = model_df[target]
identifiers = model_df[['player_id', 'season', 'player_name']] if 'player_name' in model_df.columns else model_df[['player_id', 'season']]

X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, identifiers.index, test_size=0.3, random_state=42
)

model = RandomForestRegressor(
    n_estimators=900,
    min_samples_split=11,
    min_samples_leaf=8,
    max_features=.333,
    random_state=42,
    max_depth=20,
    n_jobs=-1,
)
print(y.isna().sum(), y.describe())

model.fit(X_train, y_train)

train_r2 = model.score(X_train, y_train)
test_r2 = model.score(X_test, y_test)
print(f"Train R²: {train_r2:.3f}")
print(f"Test R²: {test_r2:.3f}")

cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2', n_jobs=-1)
print(f"Cross-validated R² scores: {cv_scores}")
print(f"Mean CV R²: {cv_scores.mean():.3f}")

importances = model.feature_importances_
plt.figure(figsize=(7, max(3, len(features_real) * 0.5)))
plt.barh(features_real, importances)
plt.xlabel("Feature Importance")
plt.title("QB Next-Season Prediction Feature Importance (previous-season stats)")
plt.show()

preds = model.predict(X)
results = pd.DataFrame({
    'player_id': identifiers.loc[X.index, 'player_id'],
    'player_name': identifiers.loc[X.index, 'player_name'] if 'player_name' in identifiers.columns else np.nan,
    'season': identifiers.loc[X.index, 'season'],
    'target_season_actual': y,
    'predicted_from_prev_season': preds,
    'error': y - preds,
    'adp': model_df['adp']
}, index=X.index)

print("\nTop predictions (sorted by predicted value):")
print(results.sort_values('predicted_from_prev_season', ascending=True).head(15))


In [ ]:
results["adp"]=results["adp"].replace(0,np.inf)
results["adp_rank"] = results.groupby("season")["adp"].rank(method="min", ascending=True)

results["predicted_rank"] = results.groupby("season")["predicted_from_prev_season"] \
                                   .rank(method="min", ascending=False)

results["rank_diff"] = results["predicted_rank"] - results["adp_rank"]

print(results.sort_values(["season", "rank_diff"],ascending=False).head(15))


In [ ]:
df_rookie=df[df["years_exp"]==0]
df_rookie=df_rookie[df_rookie["position"]=="WR"]
df_rookie=df_rookie[df_rookie["season"]>=2016]
df_rookie=df_rookie[df_rookie["season"]<=2024]
features_rookie = [                "adp","draft_ovr"
]
df_rookie=df_rookie.groupby(["player_name", "season"]).agg({
    "adp": "first",
    "draft_ovr": "first",
    "fantasy_points_ppr": "mean"
})
X = df_rookie[features_rookie]

y = df_rookie["fantasy_points_ppr"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
WR_rookie_model = RandomForestRegressor(
    n_estimators=900,
    min_samples_split=8,
    min_samples_leaf=6,
    random_state=42,
    n_jobs=-1,
    max_features=.333,)
WR_rookie_model.fit(X_train, y_train)
train_r2 = WR_rookie_model.score(X_train, y_train)
test_r2 = WR_rookie_model.score(X_test, y_test)
print(f"Train R²: {train_r2:.3f}")
print(f"Test R²: {test_r2:.3f}")
importances = WR_rookie_model.feature_importances_
plt.figure(figsize=(7, max(3, len(features_rookie) * 0.5)))
plt.barh(features_rookie, importances)
plt.xlabel("Feature Importance")
plt.title("QB Next-Season Prediction Feature Importance (previous-season stats)")
plt.show()


In [ ]:

df_real = df_real[df_real["season"] == 2024].copy()
df_real["season"] = 2025
df_real=df_real[df_real["position"]=="WR"]
years = 2025
years_list = [years] if isinstance(years, int) else list(years)
all_adp = []

for y in years_list:
    url = f"https://fantasyfootballcalculator.com/api/v1/adp/ppr?teams=12&year={y}&position=all"
    r = requests.get(url)
    r.raise_for_status()
    df_adp = pd.DataFrame(r.json()["players"])
    df_adp["season"] = y
    all_adp.append(df_adp)
    print(f"{y} — {len(df_adp)} players loaded")

adp_all = pd.concat(all_adp, ignore_index=True)
adp_all["name"] = adp_all["name"].str.strip()

adp_keep = adp_all[["name", "position", "team", "adp", "season"]].copy()
adp_keep=adp_keep[adp_keep["position"]=="WR"]

def normalize_name(s: pd.Series) -> pd.Series:
    return (
        s.fillna("")
         .str.strip()
         .str.replace(r"\s+Jr\.?$", "", regex=True)
         .str.replace(r"\s+Sr\.?$", "", regex=True)
         .str.replace(r"\s+III?$", "", regex=True)
         .str.replace(r"[^\w\s\-']", "", regex=True)
         .str.replace(r"\s+", " ", regex=True)
    )

adp_keep["player_name_merge"] = normalize_name(adp_keep["name"])
df_real["player_name_merge"] = normalize_name(df_real["player_name"])

adp_non_sum = (
    adp_keep
    .groupby(["season", "player_name_merge",] ,as_index=False)
    .agg({
        "adp": "first",
       }))


df_real = df_real.merge(adp_non_sum, on=["season", "player_name_merge"], how="outer")

rb = df_real[df_real["position"] == "WR"].copy()

num_cols = rb.select_dtypes(include="number").columns.tolist()

RB_stats = (
    rb.groupby(["player_name", "season"], as_index=False)[num_cols]
      .mean()
)

RB_adps = (
    rb.groupby(["player_name", "season"], as_index=False)
      .agg({"adp": "first", "position": "first", "team": "first"})  # pick fields you want
)


RB_2025 = RB_stats.merge(RB_adps, on=["player_name", "season"], how="left")

adp_non_sum.rename(columns={"adp": "adp_for_rank",
                            "player_name_merge":"player_name"}, inplace=True)

print(RB_stats[["player_name",  "adp"]]
      .sort_values("adp", ascending=False)
      .head(15))
rename_map = {
      "passing_yards": "passing_yards_prev",
    "fantasy_points_ppr": "fantasy_points_ppr_prev",
    "rushing_yards": "rushing_yards_prev",
    "pass_touchdown": "pass_touchdown_prev",
    "qb_dropback": "qb_dropback_prev",
    "first_down_pass": "first_down_pass_prev",
    "team_offense_snaps": "team_offense_snaps_prev",
    "touches": "touches_prev",
    "receiving_yards": "receiving_yards_prev",
    "targets": "targets_prev",
    "offense_snaps": "offense_snaps_prev",
    "receiving_touchdown": "receiving_touchdown_prev",
}
RB_2025 = RB_stats.rename(columns={k: v for k, v in rename_map.items() if k in RB_2025.columns})
RB_2025["adp"]=RB_2025["adp"].replace(0,200)
print(adp_non_sum.columns)
RB_2025.loc[RB_2025["player_name"] == "Ainias Smith", "fantasy_points_ppr_prev"] = 0
RB_2025["predicted_target_season"] = model.predict(RB_2025[features_real].values)
adp_non_sum=adp_non_sum.replace(0,200)
X_input = RB_2025[features_real]

print("NaN count per feature:\n", X_input.isna().sum())
print("Any inf?", np.isinf(X_input.to_numpy()).any())



In [ ]:
print(RB_2025)

In [ ]:
def normalize_name(name_series):
    return (
        name_series.str.strip()
        .str.replace(r"\s+Jr\.?$", "", regex=True)
        .str.replace(r"\s+Sr\.?$", "", regex=True)
        .str.replace(r"\s+III$", "", regex=True)
        .str.replace(r"\s+II$", "", regex=True)
        .str.replace(r"\s+", " ", regex=True)
    )

adp_non_sum["player_name"] = normalize_name(adp_non_sum["player_name"])
RB_2025["player_name"] = normalize_name(RB_2025["player_name"])
adp_non_sum.rename(columns={"adp": "adp_for_rank"}, inplace=True)
RB_2025 = RB_2025.merge(adp_non_sum, on=["player_name"], how="left")
RB_2025=RB_2025[["adp","player_name","fantasy_points_ppr_prev",
                 "predicted_target_season","touches_prev","adp_for_rank"]]
print(RB_2025.head(50))


In [ ]:
rookie_wrs=pd.DataFrame({"player_name":["Tetairoa McMillan",
                                        "Travis Hunter",
                                        "Mathew Golden",
                                        "Emeka Egbuka",
                                        "Jayden Higgens",
                                        "Tre' Harris",
                                        "Luther Burden III",
                                        "Kyle Williams",
                                        "Dont'e Thornton JR.",
                                        "Jack Bech",
                                        "Tory Horton",
                                        ],
                         "adp":[58.6,72.1,89.3,118.5,132.6,156,157.5,165.7,166.9,167.6,172.4,],
                         "draft_ovr":[8,2,23,19,34,39,55,69,102,58,158]})
rookie_wrs["predicted_target_season"]=WR_rookie_model.predict(rookie_wrs[features_rookie].values)
print(rookie_wrs)
rookie_rbs=rookie_wrs.rename(columns={"adp":"adp_for_rank"})

RB_2025 = pd.concat([RB_2025, rookie_wrs], ignore_index=True)


In [ ]:
vet_mask = RB_2025["fantasy_points_ppr_prev"].notna()
rookie_mask = ~vet_mask

veteran_preds = RB_2025.loc[vet_mask, "predicted_target_season"]
sigma_vet = veteran_preds.std()

rookie_preds = RB_2025.loc[rookie_mask, "predicted_target_season"]
mu_rook = rookie_preds.mean()
sigma_rook = rookie_preds.std()

def match_variance_keep_mean(x, sigma_old, sigma_new, mu_keep):
    return (x - mu_keep) / sigma_old * sigma_new + mu_keep

RB_2025.loc[rookie_mask, "predicted_target_season"] = pd.Series(
    match_variance_keep_mean(
        rookie_preds.values,
        sigma_rook,
        sigma_vet,
        mu_rook
    ),
    index=RB_2025.index[rookie_mask]
)

print("Veteran mean/std:", veteran_preds.mean(), veteran_preds.std())
print("Rookie mean/std after scaling:",
      RB_2025.loc[rookie_mask, "predicted_target_season"].mean(),
      RB_2025.loc[rookie_mask, "predicted_target_season"].std())



In [ ]:

target_mean = 10.648956
col = "predicted_target_season"

drafted_mask = (
    RB_2025["adp_for_rank"].notna()
    & np.isfinite(RB_2025["adp_for_rank"])
    & (RB_2025["adp_for_rank"] != 200)
)

current_mean = RB_2025.loc[drafted_mask, col].mean()

RB_2025.loc[drafted_mask, col] = (
    (RB_2025.loc[drafted_mask, col] - current_mean)
    + target_mean
)

print("New mean (drafted):", RB_2025.loc[drafted_mask, col].mean())
print("New std (drafted):", RB_2025.loc[drafted_mask, col].std())


In [ ]:
RB_2025.loc[RB_2025["player_name"] == "Amon-Ra St. Brown", "adp"] = 13
RB_2025.loc[RB_2025["player_name"] == "A.J. Brown", "adp"] = 19.2
RB_2025.loc[RB_2025["player_name"] == "D.J. Moore", "adp"] = 46.8


xcol = "adp"
ycol = "predicted_target_season"

plot_df = RB_2025[[xcol, ycol, "player_name"]].copy()

plot_df[xcol] = pd.to_numeric(plot_df[xcol], errors="coerce")
plot_df[ycol] = pd.to_numeric(plot_df[ycol], errors="coerce")
plot_df = plot_df.replace([np.inf, -np.inf], np.nan).dropna(subset=[xcol, ycol])
plot_df = plot_df[(plot_df[xcol] >= 1) & (plot_df[xcol] <= 400)]

if plot_df[xcol].nunique() < 2 or len(plot_df) < 3:
    print("Not enough clean data to fit a trendline.")
else:
    x = plot_df[xcol].to_numpy(dtype=float)
    y = plot_df[ycol].to_numpy(dtype=float)
    names = plot_df["player_name"].to_numpy()

    x_c = x - np.mean(x)
    y_c = y - np.mean(y)
    try:
        m, b = np.polyfit(x_c, y_c, 1)
        b = b + np.mean(y) - m * np.mean(x)
    except np.linalg.LinAlgError:
        denom = np.dot(x_c, x_c)
        if denom == 0:
            print("Cannot fit a line: zero variance in x after cleaning.")
            m, b = 0.0, np.mean(y)
        else:
            m = np.dot(x_c, y_c) / denom
            b = np.mean(y) - m * np.mean(x)

    y_pred = m * x + b
    residuals = y - y_pred
    std_dev = residuals.std(ddof=1)

    order = np.argsort(x)
    xs = x[order]
    yps = y_pred[order]

    plt.figure(figsize=(10, 7))
    plt.scatter(x, y, alpha=0.7, label="Players")
    plt.plot(xs, yps, color="red", linewidth=2, label=f"Trendline (m={m:.3f})")
    plt.fill_between(xs, yps - std_dev, yps + std_dev, color="red", alpha=0.2, label="±1σ residuals")

    for xi, yi, name in zip(x, y, names):
        plt.annotate(name, (xi, yi), fontsize=8, alpha=0.7, xytext=(3, 3), textcoords="offset points")

    plt.xlabel("Average Draft Position (ADP)")
    plt.ylabel("Projected Fantasy Points")
    plt.title("WR ADP vs Projected Fantasy Points — 2025 Season")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()
print(b)

In [ ]:
RB_2025["adp_for_rank"] = RB_2025["adp_for_rank"].fillna(200)
RB_2025.loc[RB_2025["player_name"] == "Amon-Ra St. Brown", "adp"] = 13
RB_2025.loc[RB_2025["player_name"] == "A.J. Brown", "adp"] = 19.2
RB_2025.loc[RB_2025["player_name"] == "D.J. Moore", "adp"] = 46.8


xcol = "adp"
ycol = "predicted_target_season"

plot_df = RB_2025[[xcol, ycol, "player_name"]].copy()

plot_df[xcol] = pd.to_numeric(plot_df[xcol], errors="coerce")
plot_df[ycol] = pd.to_numeric(plot_df[ycol], errors="coerce")
plot_df = plot_df.replace([np.inf, -np.inf], np.nan).dropna(subset=[xcol, ycol])
plot_df = plot_df[(plot_df[xcol] >= 1) & (plot_df[xcol] <= 400)]

if plot_df[xcol].nunique() < 2 or len(plot_df) < 3:
    print("Not enough clean data to fit a trendline.")
else:
    x = plot_df[xcol].to_numpy(dtype=float)
    y = plot_df[ycol].to_numpy(dtype=float)
    names = plot_df["player_name"].to_numpy()

    x_c = x - np.mean(x)
    y_c = y - np.mean(y)
    try:
        m, b = np.polyfit(x_c, y_c, 1)
        b = b + np.mean(y) - m * np.mean(x)
    except np.linalg.LinAlgError:
        denom = np.dot(x_c, x_c)
        if denom == 0:
            print("Cannot fit a line: zero variance in x after cleaning.")
            m, b = 0.0, np.mean(y)
        else:
            m = np.dot(x_c, y_c) / denom
            b = np.mean(y) - m * np.mean(x)

    y_pred = m * x + b
    residuals = y - y_pred
    std_dev = residuals.std(ddof=1)

    order = np.argsort(x)
    xs = x[order]
    yps = y_pred[order]

    plt.figure(figsize=(10, 7))
    plt.scatter(x, y, alpha=0.7, label="Players")
    plt.plot(xs, yps, color="red", linewidth=2, label=f"Trendline (m={m:.3f})")
    plt.fill_between(xs, yps - std_dev, yps + std_dev, color="red", alpha=0.2, label="±1σ residuals")

    for xi, yi, name in zip(x, y, names):
        plt.annotate(name, (xi, yi), fontsize=8, alpha=0.7, xytext=(3, 3), textcoords="offset points")

    plt.xlabel("Average Draft Position (ADP)")
    plt.ylabel("Projected Fantasy Points")
    plt.title("WR ADP vs Projected Fantasy Points — 2025 Season")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()
print(b)

In [ ]:
RB_2025["predicted_rank"] = RB_2025["predicted_target_season"].rank(method="min", ascending=False)

In [ ]:

RB_2025["adp_for_rank"] = RB_2025["adp_for_rank"].fillna(200)
RB_2025.loc[RB_2025["player_name"] == "Devon Achane", "adp_for_rank"] = 8.7
RB_2025.loc[RB_2025["player_name"] == "J.K. Dobbins", "adp_for_rank"] = 90.2

xcol = "predicted_rank"
ycol = "predicted_target_season"

plot_df = RB_2025[[xcol, ycol, "player_name"]].copy()

plot_df[xcol] = pd.to_numeric(plot_df[xcol], errors="coerce")
plot_df[ycol] = pd.to_numeric(plot_df[ycol], errors="coerce")
plot_df = plot_df.replace([np.inf, -np.inf], np.nan).dropna(subset=[xcol, ycol])
plot_df = plot_df[(plot_df[xcol] >= 1) & (plot_df[xcol] <= 400)]

if plot_df[xcol].nunique() < 2 or len(plot_df) < 3:
    print("Not enough clean data to fit a trendline.")
else:
    x = plot_df[xcol].to_numpy(dtype=float)
    y = plot_df[ycol].to_numpy(dtype=float)
    names = plot_df["player_name"].to_numpy()

    x_c = x - np.mean(x)
    y_c = y - np.mean(y)
    try:
        m, b = np.polyfit(x_c, y_c, 1)
        b = b + np.mean(y) - m * np.mean(x)
    except np.linalg.LinAlgError:
        denom = np.dot(x_c, x_c)
        if denom == 0:
            print("Cannot fit a line: zero variance in x after cleaning.")
            m, b = 0.0, np.mean(y)
        else:
            m = np.dot(x_c, y_c) / denom
            b = np.mean(y) - m * np.mean(x)

    y_pred = m * x + b
    residuals = y - y_pred
    std_dev = residuals.std(ddof=1)

    order = np.argsort(x)
    xs = x[order]
    yps = y_pred[order]

    plt.figure(figsize=(10, 7))
    plt.scatter(x, y, alpha=0.7, label="Players")
    plt.plot(xs, yps, color="red", linewidth=2, label=f"Trendline (m={m:.3f})")
    plt.fill_between(xs, yps - std_dev, yps + std_dev, color="red", alpha=0.2, label="±1σ residuals")

    for xi, yi, name in zip(x, y, names):
        plt.annotate(name, (xi, yi), fontsize=8, alpha=0.7, xytext=(3, 3), textcoords="offset points")

    plt.xlabel("Predicted Rank")
    plt.ylabel("Predicted Fantasy Points")
    plt.title("WR Predicted Rank vs Predicted Fantasy Points")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
RB_2025["adp_for_rank"] = RB_2025["adp_for_rank"].fillna(200)
RB_2025.loc[RB_2025["player_name"] == "Devon Achane", "adp_for_rank"] = 8.7
RB_2025.loc[RB_2025["player_name"] == "J.K. Dobbins", "adp_for_rank"] = 90.2

xcol = "predicted_rank"
ycol = "predicted_target_season"

drafted_df = RB_2025[
    RB_2025["adp_for_rank"].notna()
    & np.isfinite(RB_2025["adp_for_rank"])
    & (RB_2025["adp_for_rank"] != 200)
].copy()

plot_df = drafted_df[[xcol, ycol, "player_name"]].copy()

plot_df[xcol] = pd.to_numeric(plot_df[xcol], errors="coerce")
plot_df[ycol] = pd.to_numeric(plot_df[ycol], errors="coerce")
plot_df = plot_df.replace([np.inf, -np.inf], np.nan).dropna(subset=[xcol, ycol])
plot_df = plot_df[(plot_df[xcol] >= 1) & (plot_df[xcol] <= 400)]

if plot_df[xcol].nunique() < 2 or len(plot_df) < 3:
    print("Not enough clean data to fit a trendline.")
else:
    x = plot_df[xcol].to_numpy(dtype=float)
    y = plot_df[ycol].to_numpy(dtype=float)
    names = plot_df["player_name"].to_numpy()

    x_c = x - np.mean(x)
    y_c = y - np.mean(y)
    try:
        m, b = np.polyfit(x_c, y_c, 1)
        b = b + np.mean(y) - m * np.mean(x)
    except np.linalg.LinAlgError:
        denom = np.dot(x_c, x_c)
        if denom == 0:
            print("Cannot fit a line: zero variance in x after cleaning.")
            m, b = 0.0, np.mean(y)
        else:
            m = np.dot(x_c, y_c) / denom
            b = np.mean(y) - m * np.mean(x)

    y_pred = m * x + b
    residuals = y - y_pred
    std_dev = residuals.std(ddof=1)

    order = np.argsort(x)
    xs = x[order]
    yps = y_pred[order]

    plt.figure(figsize=(10, 7))
    plt.scatter(x, y, alpha=0.7, label="Players")
    plt.plot(xs, yps, color="red", linewidth=2, label=f"Trendline (m={m:.3f})")
    plt.fill_between(xs, yps - std_dev, yps + std_dev, color="red", alpha=0.2, label="±1σ residuals")

    for xi, yi, name in zip(x, y, names):
        plt.annotate(name, (xi, yi), fontsize=8, alpha=0.7, xytext=(3, 3), textcoords="offset points")

    plt.xlabel("Predicted Rank")
    plt.ylabel("Predicted Fantasy Points")
    plt.title("WR Predicted Rank vs Predicted Fantasy Points — 2025 Season (Drafted Players Only)")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
RB_2025.sort_values("predicted_target_season", ascending=True).head(40)
print(RB_2025.loc[RB_2025["player_name"] == "Mathew Golden"])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
RB_2025.to_csv("/content/drive/MyDrive/WR_2025.csv")